In [18]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG19
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping

from PIL import Image #importing the python image library for PIL
import os,cv2
import random
import numpy as np
import pandas as pd
from scipy import misc 
from sklearn.model_selection import train_test_split

import tensorflow.keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential

In [19]:
root_dir = os.path.abspath('.')
data_dir_test = './datasets/misctest'
data_dir = './datasets/misctrain'
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir_test, 'test.csv'))

In [20]:
X  = train.drop(columns ='Class' )
y = train.Class

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify=y)

In [22]:
from scipy.misc import imresize
from scipy.misc import imread

temp = []
for img_name in X_train.ID:
    img_path = os.path.join(data_dir, 'Train', img_name)
    img = imread(img_path)
    img = imresize(img, (100, 100))
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)

X_train = np.stack(temp)

/Users/payalchodha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
/Users/payalchodha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [23]:
from scipy.misc import imresize
from scipy.misc import imread

temp = []
for img_name in X_test.ID:
    img_path = os.path.join(data_dir, 'Train', img_name)
    img = imread(img_path)
    img = imresize(img, (100,100))
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)

X_test = np.stack(temp)


/Users/payalchodha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
/Users/payalchodha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [24]:


# The current range of X_train and X_test is 0 to 255.
# The code below is equivalent to X_train = X_train / 255.
# This scales each value to be between 0 and 1.
X_train /= 255
X_test /= 255

In [25]:
# Reshape each image to be 100 x 100 x 3.
X_train = X_train.reshape(X_train.shape[0], 100, 100, 3)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 3)

In [27]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
# y = np_utils.to_categorical(y)
# Change y_train.
y_train = np_utils.to_categorical(y_train, 3)
# y_test = np_utils.to_categorical(y_test, 3)

In [28]:
y_test = lb.fit_transform(y_test)
y_test = np_utils.to_categorical(y_test, 3)

In [2]:
#setting the base VGG19
conv_base = VGG19(weights = 'imagenet', 
                 include_top = False,
                 input_shape = (100, 100, 3))

80142336/80134624 [==============================] - 2s 0us/step


In [30]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [31]:
model = models.Sequential()
model.add(conv_base)
#defining the final layers
model.add(Conv2D(filters = 64,
                     kernel_size=(3,3),
                     activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [32]:

# Compile model
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-5),
                  metrics=['accuracy'])


In [33]:
early_stop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0, #if it ticks up by a little bit(i consider it as an increase)
                          patience = 3) #number of epochs with no improvement after which training will be stopped

In [ ]:
# Fit model on training data
history = model.fit(X_train,
                        y_train,
                        batch_size=128,
                        validation_data=(X_test, y_test),
                        epochs=20,
                        callbacks=[early_stop],
                        verbose=1)